In [27]:
import mysql.connector
import yaml
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

with open('config.yml', 'r') as c:
    config = yaml.load(c)
cnx = mysql.connector.connect(user=config["database"]["user"],
                              password=config["database"]["password"],
                              host=config["database"]["host"],
                              database=config["database"]["db"])
cursor = cnx.cursor()

def execute_query(q):
    cursor.execute(q)
    return cursor.fetchall()

def plot_bar_chart(rows, name, color):
    x = []
    y = []
    for row in rows:
        x.append('{}-{}'.format(row[0], row[1]))
        y.append(row[2])
    data = [
        go.Bar(x=x,
               y=y,
               name=name,
               marker=dict(color=color))
    ]
    iplot(data)

In [34]:
query_npd_comments_by_month = (
    'SELECT year(created_time) year, month(created_time) month, count(*) count '
    'FROM comment '
    'WHERE page=1 AND year(created_time)>=2011 '
    'GROUP BY year(created_time), month(created_time)'
)
rows_npd_comments_by_month = execute_query(query_npd_comments_by_month)
plot_bar_chart(rows_npd_comments_by_month, 'NPD', 'rgb(224, 6, 34)')

In [35]:
query_afd_comments_by_month = (
    'SELECT year(created_time) year, month(created_time) month, count(*) count '
    'FROM comment '
    'WHERE page=22 AND year(created_time)>=2011 '
    'GROUP BY year(created_time), month(created_time)'
)
rows_afd_comments_by_month = execute_query(query_afd_comments_by_month)
plot_bar_chart(rows_afd_comments_by_month, 'AfD', 'rgb(0, 158, 224)')

In [36]:
rows_npd_comments_by_month

x = []
y = []
for row in rows_npd_comments_by_month:
    x.append('{}-{}'.format(row[0], row[1]))
    y.append(row[2])


x2 = []
y2 = []
for row in rows_afd_comments_by_month:
    x2.append('{}-{}'.format(row[0], row[1]))
    y2.append(row[2])


data = [go.Bar(x=x2,
               y=y2,
               name='AfD',
               marker=dict(color='rgb(0, 158, 224)')), 
        go.Bar(x=x,
               y=y,
               name='NPD',
               marker=dict(color='rgb(224, 6, 34)'))]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)